In [1]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
WEAVIATE_URL="https://frqjst3xshkqyw7ayhy6a.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="eQJokAVcxKpPbEXRjr5E2tSi1lvXen5JAsgT"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [3]:
# Setup vector store
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)


# client = weaviate.connect_to_local(
#                      host="192.168.100.125",
#                         port=8080,
#                         grpc_port=50051
#                 )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/weaviate/warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(


In [4]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [5]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=10,
    alpha=0.3,
)

In [6]:
question = "Luật giao thông quy định như thế nào về tổng số điểm của giấy phép lái xe và quy định về việc cộng điểm?"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : cá_nhân 
 mức phạt : phạt tiền từ 10.000.000 đồng đến 15.000.000 đồng 
 nội_dung vi_phạm : ( cá_nhân ) sử_dụng tạm_thời lòng_đường , vỉa_hè vào mục_đích khác ( theo quy_định phải có giấy_phép ) mà không có giấy_phép hoặc có giấy_phép nhưng đã hết giá_trị sử_dụng hoặc thực_hiện không đúng nội_dung ghi trong giấy_phép .

Metadata: {'category': 'Cá nhân', 'fine_amount': 'Phạt tiền từ 10.000.000 đồng đến 15.000.000 đồng', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: Cá nhân\nMức phạt: Phạt tiền từ 10.000.000 đồng đến 15.000.000 đồng\nNội dung vi phạm: (Cá nhân) Sử dụng tạm thời lòng đường, vỉa hè vào mục đích khác (theo quy định phải có giấy phép) mà không có giấy phép hoặc có giấy phép nhưng đã hết giá trị sử dụng hoặc thực hiện không đúng nội dung ghi trong giấy phép.'}

Score: 1.0

Result 2:

Text: loại phương_tiện : tổ_chức 
 mức phạt : phạt tiền từ 20.000.000 đồng đến 30.000.000 đồng 
 nội_dung vi_phạm : ( tổ_chức ) sử_dụng tạm_thời

In [7]:
client.collections.delete_all()